# Match do emprego

Apresenta o préprocessamento para os datasets de vagas e de candidatos, efetuando a limpeza dos campos, retirada de valores nulos e transformação de atributos categóricos em numéricos.
Para o dataset de candidatos foram carregados apenas os 2000 primeiros registros de candidados, em função do grande volume de dados

In [21]:
# Verificar se é necessário instalar a biblioteca abaixo
!pip install openpyxl

You should consider upgrading via the 'C:\projetos\env\pesquisa_pref\Scripts\python.exe -m pip install --upgrade pip' command.


In [22]:
import pandas as pd
import numpy as np

In [23]:
# df_vagas = pd.read_csv('data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_VAGAS_PR.csv', encoding = "ISO-8859-1",sep=';')
df_vagas = pd.read_excel('data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_VAGAS_BA.xlsx')
pd.set_option('max_colwidth', None)
df_vagas.head()

,CODIGO_OCUPACAO,TITULO_OCUPACAO,OBSERVACOES_OCUPACAO,CODIGO_MUNICIPIO_IBGE,CEP,VALOR_SALARIO,QTD_VAGAS,TIPO_CONTRATACAO,QTD_EXPERIENCIA,ESCOLARIDADE,REQUERIDO_DEFICIENCIA,DEFICIENCIAS
0,516120,Manicure,NaN,291080,44100000,0.0,1,3 - Permanente,6,7 - Médio Completo,0,0 - Surdez bilateral parcial - Auditiva|8 - Nanismo - Física
1,411005,Auxiliar de escritório,Trabalho a ser executado sera o de preparacao de documentos para digitalizacao e geracao de imagens,291955,47850000,0.0,80,3 - Permanente,0,7 - Médio Completo,0,0 - Surdez bilateral parcial - Auditiva|1 - Surdez bilateral total - Auditiva
2,783215,Chapa (movimentador de mercadoria),NaN,291072,45826000,880.0,1,3 - Permanente,0,7 - Médio Completo,0,0 - Surdez bilateral parcial - Auditiva|2 - Amputação - Física|5 - Um membro Inferior - Física|7 - Um membro superior - Física|11 - Parcial - Física| 13 - Baixa Visão - Visual
3,521110,Vendedor interno,realizar vendas de cotas de consorcio por telefone,291980,1014000,0.0,6,3 - Permanente,6,7 - Médio Completo,0,0 - Surdez bilateral parcial - Auditiva|1 - Surdez bilateral total - Auditiva|10 - Paralisia Cerebral - Física|12 - Mental / Intelectual - Mental| 13 - Baixa Visão - Visual|14 - Cegueira - Visual
4,524105,Vendedor porta a porta,NaN,292740,40301146,0.0,3,3 - Permanente,0,5 - Fundamental Completo,0,0 - Surdez bilateral parcial - Auditiva|4 - Membros Inferiores - Física|5 - Um membro Inferior - Física|7 - Um membro superior - Física|8 - Nanismo - Física| 13 - Baixa Visão - Visual


### Pré-processamento vagas 
- tipo_contratacao e escolaridade: substituindo valores textuais para numéricos
- qtd_experiencia: preenchendo campos vazios com zero e convertendo campo float para int**
- n_deficiencia: 1 sim, 0 não - usado para indicar se a vaga é para portador de deficiência
- cd_municipio_ibge: indica o município para a vaga


In [24]:
# Remove as colunas que não serão utilizadas
df_vagas=df_vagas.drop(['DEFICIENCIAS','CEP','OBSERVACOES_OCUPACAO','TITULO_OCUPACAO','QTD_VAGAS'], axis=1)

df_vagas.rename(columns={'CODIGO_OCUPACAO': 'cod_ocupacao', 'VALOR_SALARIO': 'salario', 'TIPO_CONTRATACAO': 'tipo_contratacao',
'QTD_EXPERIENCIA': 'qtd_experiencia', 'ESCOLARIDADE': 'escolaridade','CODIGO_MUNICIPIO_IBGE':'cd_municipio_ibge','REQUERIDO_DEFICIENCIA':'n_deficiencia'}, inplace=True)

In [25]:
def get_escolaridade_index(df_escolaridade):
    vet_escolaridade=['-1 - Não Identificado','0 - Nenhum','1 - Analfabeto','10 - Fundamental Incompleto', '5 - Fundamental Completo','6 - Médio Incompleto','7 - Médio Completo', '8 - Superior Incompleto', 'Superior Completo','13 - Especialização','11 - Mestrado', '12 - Doutorado']
    #vet_escolaridade=['Não Identificado','Nenhum','Analfabeto','Fundamental Incompleto','Fundamental Completo','Médio Incompleto','Médio Completo','Superior Incompleto','Superior Completo','Mestrado', 'Doutorado','Especialização']
    df_esc=[]
    for i in range(len(df_escolaridade)):
      index=-1  
      for j in range(len(vet_escolaridade)):
        if(vet_escolaridade[j]==str(df_escolaridade[i]).rstrip(' ')):
          index=j-1
      df_esc.append(index)
    return df_esc  

In [26]:

# Tipo contratacao
df_vagas.loc[df_vagas['tipo_contratacao'] == '0 - Estágio', 'tipo_contratacao']     = 0
df_vagas.loc[df_vagas['tipo_contratacao'] == '1 - Aprendiz', 'tipo_contratacao']    = 1 
df_vagas.loc[df_vagas['tipo_contratacao'] == '2 - Temporária', 'tipo_contratacao']  = 2
df_vagas.loc[df_vagas['tipo_contratacao'] == '3 - Permanente', 'tipo_contratacao']  = 3

# Escolaridade
df_vagas['escolaridade']=get_escolaridade_index(df_vagas['escolaridade'])

# Qtd. Experiência
df_vagas.fillna(0, inplace=True)
df_vagas.qtd_experiencia =df_vagas.qtd_experiencia.astype(int)

df_vagas['cod_ocupacao'] = df_vagas.cod_ocupacao.astype(np.int64)
df_vagas['salario'] = df_vagas.salario.astype(np.int64)

df_vagas.to_excel('data/vagas-ba.xlsx')
df_vagas

,cod_ocupacao,cd_municipio_ibge,salario,tipo_contratacao,qtd_experiencia,escolaridade,n_deficiencia
0,516120,291080,0,3,6,5,0
1,411005,291955,0,3,0,5,0
2,783215,291072,880,3,0,5,0
3,521110,291980,0,3,6,5,0
4,524105,292740,0,3,0,3,0
...,...,...,...,...,...,...,...
71,514320,291005,0,3,0,4,0
72,142345,293135,0,3,6,7,0
73,411010,293135,0,0,6,6,0
74,721215,293135,0,3,6,5,0


### Pré-processamento candidatos
- deficiencia: 0 (não), 1 (sim)
- escolaridade: domínio de -1 (Não Identificado) a 8 (Doutorado)

In [27]:
#Carrega os primeiros 2000 candidatos
df_candidatos = pd.read_excel('data/D_ETL_IMO_EXTRACAO_SINE_ABERTO_TRABALHADORES_BA2000.xlsx')

In [28]:
df_candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   NACIONALIDADE               1999 non-null   object
 1   DEFICIENCIAS                26 non-null     object
 2   BAIRRO                      1130 non-null   object
 3   CEP                         1999 non-null   int64 
 4   CODIGO_MUNICIPIO_IBGE       1999 non-null   int64 
 5   NOME_MUNICIPIO              1999 non-null   object
 6   UF                          1999 non-null   object
 7   ESCOLARIDADE                1999 non-null   object
 8   ESTUDANTE                   1999 non-null   object
 9   CURSOS_PROFISSIONALIZANTES  451 non-null    object
 10  GRADUACOES                  164 non-null    object
 11  POS_GRADUACOES              5 non-null      object
 12  IDIOMAS                     83 non-null     object
 13  HABILITACAO                 1999 non-null   obje

In [29]:
import re 

def getOcupacoes(txtOcupacoes):
    pattern_cod_ocupacao = '[\d]{6}' 

    ocupacoes = txtOcupacoes.split('|')
    cod_ocupacao = []

    for partOcupacao in ocupacoes:
        print('Ocupação pretendida: ' + partOcupacao)
        cod_ocupacao += re.findall(pattern_cod_ocupacao, partOcupacao)

    return cod_ocupacao


In [30]:
#Visualiza domínio de CURSOS_PROFISSIONALIZANTES
df_candidatos['CURSOS_PROFISSIONALIZANTES'].unique()

array([nan, 'EMPILHADEIRA |INFORMATICA BASICA',
       'atendimento ao publico|verificar escolaridade sem comprovação!!!!!!!!!!!!!!!',
       'ATENDENTE DE FARMACIA|ATENDENTE HOSPITALAR|DIGITAÇÃO|EDIÇÃO DE AUDIO E VIDEO|INFORMATICA',
       'ensino medio-com comprovacao', 'direcao defensiva|moop',
       'auto cad 2d', 'capacitação profissional|rotinas administrativas',
       'escolaridade comprovada', 'instrumentação',
       'capacitação para condutor profissional', 'Vigilante.. Cftv..',
       'administracao EM GERAL|informatica basica',
       'atendimento ao pulblico|recepçao|tecnicas em vendas',
       'auxiliar de produção|boas praticas de fabricacao|informatica basica',
       'ATENDENTE DE BAL|ATENDIMENTO AO PÚBLICO|AUXILIAR ADM|INFORMÁTICA BÁSICA',
       'cftv  11h/21|técnico em mecânica - cursando|vigilante 200h/21',
       'ELETRICISTA|MANUTENÇÃO DE MICROS|NR10', 'informatica basica',
       'ELETRICISTA FORÇA E CONTROLE (SENAI)|NORMA REGULAMENTADORA NR 10 (SENAI)|SEGURAN

In [31]:
#Função para calcular o número de itens
def get_qtd_itens(df):
    df_new=[]
    for i in range(len(df)):
      if(pd.isna(df[i])):
        df_new.append(0)
      else:
        texto=str(df[i]).split("|") 
        df_new.append(len(texto)) 
    return df_new


In [32]:

#Quantifica os cursos de graduação, profissionalizantes e total de idiomas
df_candidatos['N_GRADUACOES']=get_qtd_itens(df_candidatos['GRADUACOES'])
df_candidatos['N_CURSOS']=get_qtd_itens(df_candidatos['CURSOS_PROFISSIONALIZANTES'])
df_candidatos['N_IDIOMAS']=get_qtd_itens(df_candidatos['IDIOMAS'])

df_candidatos['N_ESCOLARIDADE']=get_escolaridade_index(df_candidatos['ESCOLARIDADE'])

In [33]:
#Preenche o campo MUNICIPIOS_INTERESSE nulo com o CODIGO_MUNICIPIO_IBGE do candidato
df_candidatos.MUNICIPIOS_INTERESSE.fillna(df_candidatos.CODIGO_MUNICIPIO_IBGE, inplace=True)

In [34]:
df_candidatos.DEFICIENCIAS.fillna("0", inplace=True)

In [35]:
# Deficiência
#df_candidatos.loc[df_candidatos['DEFICIENCIAS'] != "0", 'N_DEFICIENCIA'] = "1"
df_candidatos.DEFICIENCIAS.fillna("0", inplace=True)
df_candidatos.loc[df_candidatos['DEFICIENCIAS']!='0', 'DEFICIENCIAS'] = '1'

In [36]:
df_candidatos['N_DEFICIENCIA']=df_candidatos['DEFICIENCIAS']

In [37]:
#Remove as colunas que não serão utilizadas exceto MUNICIPIOS_INTERESSE
df_candidatos=df_candidatos.drop(['NACIONALIDADE','DEFICIENCIAS', 'BAIRRO','CEP','NOME_MUNICIPIO','UF','ESTUDANTE','CURSOS_PROFISSIONALIZANTES','GRADUACOES','IDIOMAS','HABILITACAO','VEICULOS','DISP_VIAJAR','DISP_AUSENTAR_DOMIC','DISP_DORMIR_EMP','POS_GRADUACOES','ESCOLARIDADE','CODIGO_MUNICIPIO_IBGE'], axis=1)

In [38]:
df_candidatos.N_DEFICIENCIA.unique()

array(['0', '1'], dtype=object)

In [39]:
#Retira do dataset os perfis que não possuem pretenção de cargos
df_candidatos=df_candidatos[df_candidatos['PRETENSOES'].notna()]

In [40]:
#Resetando o índice 
df_candidatos.reset_index(drop=True,inplace=True)

In [41]:
# Trata o campo de texto de pretensões transformando-o de texto livre para vetor
df_new=[]
df_id=[]
df=df_candidatos['PRETENSOES']
for i in range(len(df)):
    df_id.append(i+1)  # cria um ID para o candidato
    vet_pret=[]
    texto=df[i].split("|") 
    for j in range(len(texto)): #len(texto)
      texto[j]=texto[j][0:6]
      texto[j]=texto[j].replace('-', '')
      texto[j]=texto[j].replace('M', '')
      texto[j]=texto[j].replace('a', '')
      vet_pret.append(texto[j]) 
    df_new.append(vet_pret)  
df_candidatos['ID_CANDIDATO']=df_id    
df_candidatos['CBO_PRETENSAO']=df_new    



In [42]:
# Remove a coluna de pretensões
df_candidatos=df_candidatos.drop(['PRETENSOES'], axis=1)

In [43]:
df_candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1944 entries, 0 to 1943
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   MUNICIPIOS_INTERESSE  1944 non-null   object
 1   N_GRADUACOES          1944 non-null   int64 
 2   N_CURSOS              1944 non-null   int64 
 3   N_IDIOMAS             1944 non-null   int64 
 4   N_ESCOLARIDADE        1944 non-null   int64 
 5   N_DEFICIENCIA         1944 non-null   object
 6   ID_CANDIDATO          1944 non-null   int64 
 7   CBO_PRETENSAO         1944 non-null   object
dtypes: int64(5), object(3)
memory usage: 121.6+ KB


In [44]:
#Quebra os vetores de Pretenções duplicando as linhas de perfil do candidato
df_new=[]
for row in df_candidatos.itertuples(index=False):
  for j in range(len(row.CBO_PRETENSAO)):
    n_row=(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7][j])
    df_new.append(list(n_row))

In [45]:
df_candidatos = pd.DataFrame(df_new,columns=['municipio_interesse','n_graduacoes','n_cursos','n_idiomas','n_escolaridade','n_deficiencia','id_cadidato','cbo_pretensao'])
df_candidatos = df_candidatos.astype({'municipio_interesse': str, 'n_graduacoes':int, 'n_cursos':int, 'n_idiomas': int, 'n_escolaridade': int, 'n_deficiencia': int, 'id_cadidato': int, 'cbo_pretensao': int })

In [46]:
df_candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6319 entries, 0 to 6318
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   municipio_interesse  6319 non-null   object
 1   n_graduacoes         6319 non-null   int32 
 2   n_cursos             6319 non-null   int32 
 3   n_idiomas            6319 non-null   int32 
 4   n_escolaridade       6319 non-null   int32 
 5   n_deficiencia        6319 non-null   int32 
 6   id_cadidato          6319 non-null   int32 
 7   cbo_pretensao        6319 non-null   int32 
dtypes: int32(7), object(1)
memory usage: 222.3+ KB


In [47]:
# Trata o campo de texto de MUNICIPIOS_INTERESSE transformando-o de texto livre para vetor
df_new=[]
df=df_candidatos['municipio_interesse']
for i in range(len(df)):
    vet_pret=[]
    texto=df[i].split("|") 
    for j in range(len(texto)):#len(texto)
      vet_pret.append(texto[j]) 
    df_new.append(vet_pret)  
df_candidatos['cd_ibge_municipio_interesse']=df_new   
df_candidatos=df_candidatos.drop(['municipio_interesse'], axis=1)


In [48]:
#Quebra os vetores de cd_ibge_municipio_interesse duplicando as linhas de perfil do candidato
df_new=[]
for row in df_candidatos.itertuples(index=False):
  for j in range(len(row.cd_ibge_municipio_interesse)):
    n_row=(row[5],row[7][j],row[6],row[0],row[1],row[2],row[3],row[4])
    df_new.append(list(n_row))

In [49]:
#renomeia os campos de acordo com a nova organização
df_candidatos = pd.DataFrame(df_new,columns=['id_cadidato','cd_ibge_municipio_interesse','cbo_pretensao','n_graduacoes','n_cursos','n_idiomas','n_escolaridade','n_deficiencia'])

In [50]:

df_candidatos.head(100)

,id_cadidato,cd_ibge_municipio_interesse,cbo_pretensao,n_graduacoes,n_cursos,n_idiomas,n_escolaridade,n_deficiencia
0,1,290570,414140,0,0,0,-1,0
1,1,290570,521110,0,0,0,-1,0
2,1,290570,716610,0,0,0,-1,0
3,1,290570,717020,0,0,0,-1,0
4,1,290570,784205,0,0,0,-1,0
...,...,...,...,...,...,...,...,...
95,29,292740,422315,0,1,0,-1,0
96,29,292740,521110,0,1,0,-1,0
97,30,292100,513505,0,0,0,-1,0
98,31,292950,514320,0,1,0,-1,0


In [51]:
 # salva o dataset de candidatos para utilização na fase de avaliação dos hiperparâmetros
 df_candidatos.to_excel('data/candidatos-ba.xlsx')
 df_candidatos

,id_cadidato,cd_ibge_municipio_interesse,cbo_pretensao,n_graduacoes,n_cursos,n_idiomas,n_escolaridade,n_deficiencia
0,1,290570,414140,0,0,0,-1,0
1,1,290570,521110,0,0,0,-1,0
2,1,290570,716610,0,0,0,-1,0
3,1,290570,717020,0,0,0,-1,0
4,1,290570,784205,0,0,0,-1,0
...,...,...,...,...,...,...,...,...
6694,1944,291400,422105,0,3,0,-1,0
6695,1944,291400,521125,0,3,0,-1,0
6696,1944,291400,764005,0,3,0,-1,0
6697,1944,291400,764205,0,3,0,-1,0


In [52]:
#Medidas descritivas
df_candidatos.describe()

,id_cadidato,cbo_pretensao,n_graduacoes,n_cursos,n_idiomas,n_escolaridade,n_deficiencia
count,6699.000000,6699.000000,6699.000000,6699.000000,6699.000000,6699.000000,6699.000000
mean,977.617256,554511.365577,0.120316,1.082400,0.099866,-0.479325,0.014480
std,565.059176,170363.777254,0.379967,2.110649,0.450306,1.973544,0.119466
min,1.000000,111220.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
25%,490.000000,422105.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
50%,971.000000,517410.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
75%,1478.000000,717020.000000,0.000000,1.000000,0.000000,-1.000000,0.000000
max,1944.000000,992225.000000,3.000000,15.000000,5.000000,7.000000,1.000000


In [53]:
#Contagem de campos nulos
df_candidatos.isna().sum()

id_cadidato                    0
cd_ibge_municipio_interesse    0
cbo_pretensao                  0
n_graduacoes                   0
n_cursos                       0
n_idiomas                      0
n_escolaridade                 0
n_deficiencia                  0
dtype: int64